<a href="https://colab.research.google.com/github/MiinJng/Test_ML/blob/main/5.3%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랜덤 포레스트
- 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징이다.
    - 부트스트랩 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미한다.
    - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.
    - 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다.
    - 4개의 특성이 있다면, 노드마다 2개를 랜덤하게 선택하여 사용한다.
- 회귀 모델인 RandomForestRegressor는 전체 특성을 사용한다.
- 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다.
- 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.
- 회귀일 때는 단순히 각 트리의 예측을 평균한다.
- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.
- 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 criterion, max_length, max_features, min_sample_split, min_impurity_decrease, min_samples_leaf 등의 매개변수를 제공
- 특성 중요도도 계산(랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리
- 랜덤 포레스트와 매우 비슷하게 동작
- 기본적으로 100개의 결정 트리를 훈련
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용
- 랜덤 포레스트와 엑스트라 트리의 차이점 - 부트스트랩 샘플을 사용하지 않음
    - 각 결정 트리를 만들 때 전체 훈련 세트를 사용함
    - 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아닌, 무작위로 분할함
    - DecisionTreeClassifier의 splitter=‘random’인 결정 트리
- 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음
- 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함
- 하지만 랜덤하게 노드를 분할하기 때문에, 빠른 계산 속도가 장점임

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용함
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고, 일반적으로 높은 일반화 성능을 기대할 수 있음
- 경사 하강법을 사용하여 트리 앙상블을 추가함
    - 분류에서는 로지스틱 손실 함수를 사용, 회귀에서는 평균 제곱 오차 함수를 사용
    - 4장의 경사 하강법은 손실 함수를 산으로 정의하고, 가장 낮은 곳을 찾아 내려오는 과정으로 설명함
    - 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것
    - 손실 함수의 낮은 곳으로 천천히 조금씩 이동 = 깊이가 얕은 트리를 사용
- 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함
- 학습률 매개변수로 속도를 조절함

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 부스팅
- 입력 특성을 256개의 구간으로 나눈다.
- 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다
- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용한다.
    - 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.
- 사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier이다.
    - 기본 매개변수에서 안정적인 성능을 얻을 수 있다.
    - **트리의 개수를 지정**하는데, n_estimators 대신에 부스팅 반복 횟수를 지정하는 **max_iter**를 사용함
- **sklearn.experimental** 패키지 아래에 있는 **enable_hist_gradient_boosting** 모듈을 임포트 해야함

In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

## XGBoost

In [14]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


## LightGBM

In [15]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
